In [ ]:
import numpy as np
from view_sampler import *
from algs import *

import loss_funcs
from evaluate import eval_funcs

from evaluate.eval_log import EvalLog
from evaluate.evaluator import Evaluator
from evaluate.dataset import Dataset
from utils.visualize import *

import mealpy

plotter = SearchPlotter(update_freq=100, history=None, alpha=0.5)

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.3), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/hammer/world.xml", cam_config, simulation_time=0)
sim_viewer = ViewSampler("data/hammer/world_sim.xml", cam_config)

loss_func = loss_funcs.WeightedSum(loss_funcs.IOU(), loss_funcs.RMSE())

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.OriginalSA()),  # classic
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.SA.SwarmSA(step_size=np.pi / 100)),  # pretty bad, classic
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.OriginalPSO()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.PSO.HPSO_TVAC()),  # very good
    MealAlgorithm(
        sim_viewer, loss_func, mealpy.music_based.HS.OriginalHS(c_r=0.5, pa_r=0.25)
    ),  # interesting but pretty bad
    MealAlgorithm(sim_viewer, loss_func, mealpy.system_based.GCO.DevGCO()),  # decent
    MealAlgorithm(sim_viewer, loss_func, mealpy.QSA.OriginalQSA()),  # seems pretty good
    MealAlgorithm(sim_viewer, loss_func, mealpy.BRO.DevBRO()),
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.CGO.OriginalCGO()),  # weird algorithm
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.CircleSA.OriginalCircleSA(c_factor=0.8)),  # interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.HC.OriginalHC()),  # classic, pretty good
    MealAlgorithm(sim_viewer, loss_func, mealpy.math_based.SCA.DevSCA()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.EP.LevyEP(bout_size=0.15)),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.DE.OriginalDE()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.FPA.OriginalFPA()),  # very good
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.GA.EliteMultiGA()),  # decent
    MealAlgorithm(sim_viewer, loss_func, mealpy.evolutionary_based.MA.OriginalMA()),  # very interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.CA.OriginalCA()),  # interesting
    MealAlgorithm(sim_viewer, loss_func, mealpy.human_based.FBIO.DevFBIO()),  # very very good
    MealAlgorithm(
        sim_viewer, loss_func, mealpy.human_based.ICA.OriginalICA(revolution_prob=0.4, empire_count=7)
    ),  # very good and cool
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.CDO.OriginalCDO()),  # legendary name
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EFO.DevEFO()),  # very good, fast convergance
    MealAlgorithm(sim_viewer, loss_func, mealpy.physics_based.EO.ModifiedEO()),  # promosing
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.ARO.OriginalARO()),  # promising
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.BeesA.ProbBeesA()),  # interesting
    MealAlgorithm(
        sim_viewer, loss_func, mealpy.swarm_based.DO.OriginalDO()
    ),  # interesting but pretty bad, builds a cube
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.EHO.OriginalEHO()),  # cool algorithm
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.MRFO.OriginalMRFO()),  # pretty good
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.MSA.OriginalMSA()),  # pretty good
]

algorithms: list[MealAlgorithm] = [
    MealAlgorithm(sim_viewer, loss_func, mealpy.swarm_based.SCSO.OriginalSCSO()),
]

eval_func = eval_funcs.XorDiff(0.1)
evaluator = Evaluator(world_viewer, sim_viewer, eval_func=eval_func)

dataset = Dataset.create_random(location=(0, 1.3, 0.3), num_samples=3, seed=1)
# dataset.visualize(sim_viewer)
run_config = MealRunConfig(time_limit=15)

for alg in algorithms:
    # log = EvalLog(alg)
    eval_losses = evaluator.evaluate(alg, run_config, dataset, plotter=plotter)
    # log.save("logs/")
    print(f"{alg.get_name()}: {eval_losses}")

In [ ]:
world_viewer.close()
sim_viewer.close()
plotter.close()